In [1]:
import requests
import json
import datetime
import time
import yaml

In [2]:
with open("C:\koreainvestment-autotrade-main\config.yaml", encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
    
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
ACCESS_TOKEN = ""
CANO = _cfg['CANO']
ACNT_PRDT_CD = _cfg['ACNT_PRDT_CD']
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']
URL_BASE = _cfg['URL_BASE']

In [3]:
def send_message(msg):
    """디스코드 메세지 전송"""
    now = datetime.datetime.now()
    message = {"content": f"[{now.strftime('%Y-%m-%d %H:%M:%S')}] {str(msg)}"}
    requests.post(DISCORD_WEBHOOK_URL, data=message)
    print(message)

In [4]:
def get_access_token():
    """토큰 발급"""
    headers = {"content-type": "application/json"}
    body = {"grant_type": "client_credentials",
            "appkey": APP_KEY,
            "appsecret": APP_SECRET}
    PATH = "oauth2/tokenP"
    URL = f"{URL_BASE}/{PATH}"
    res = requests.post(URL, headers=headers, data=json.dumps(body))
    ACCESS_TOKEN = res.json()["access_token"]
    return ACCESS_TOKEN

In [5]:
def hashkey(datas):
    """암호화"""
    PATH = "uapi/hashkey"
    URL = f"{URL_BASE}/{PATH}"
    headers = {
        'content-Type': 'application/json',
        'appKey': APP_KEY,
        'appSecret': APP_SECRET,
    }
    res = requests.post(URL, headers=headers, data=json.dumps(datas))
    hashkey = res.json()["HASH"]
    return hashkey

In [6]:
def get_current_price(code):
    """현재가 조회"""
    PATH = "uapi/domestic-stock/v1/quotations/inquire-price"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type": "application/json",
               "authorization": f"Bearer {ACCESS_TOKEN}",
               "appKey": APP_KEY,
               "appSecret": APP_SECRET,
               "tr_id": "FHKST01010100"}
    params = {
        "fid_cond_mrkt_div_code": "J",
        "fid_input_iscd": code,
    }
    res = requests.get(URL, headers=headers, params=params)
    return int(res.json()['output']['stck_prpr'])

In [7]:
def get_target_price(code, select):
    """변동성 돌파 전략으로 매수 목표가 조회"""
    PATH = "uapi/domestic-stock/v1/quotations/inquire-daily-price"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type": "application/json",
               "authorization": f"Bearer {ACCESS_TOKEN}",
               "appKey": APP_KEY,
               "appSecret": APP_SECRET,
               "tr_id": "FHKST01010400"}
    params = {
        "fid_cond_mrkt_div_code": "J",
        "fid_input_iscd": code,
        "fid_org_adj_prc": "1",
        "fid_period_div_code": "D"
    }
    res = requests.get(URL, headers=headers, params=params)
    stck_oprc = int(res.json()['output'][0]['stck_oprc'])  # 오늘 시가
    stck_hgpr = int(res.json()['output'][1]['stck_hgpr'])  # 전일 고가
    stck_lwpr = int(res.json()['output'][1]['stck_lwpr'])  # 전일 저가
    stck_range = stck_hgpr - stck_lwpr
    
    if(select == "0"): 
        k = 0.13
        print(k)
    elif(select == "1"):
        k = 0.5
        print(k)
    else:
        k = 0.88
        print(k)
      
    target_price = stck_oprc + (stck_range * k)
    
    return target_price

In [8]:
def get_stock_balance():
    """주식 잔고조회"""
    PATH = "uapi/domestic-stock/v1/trading/inquire-balance"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type": "application/json",
               "authorization": f"Bearer {ACCESS_TOKEN}",
               "appKey": APP_KEY,
               "appSecret": APP_SECRET,
               "tr_id": "TTTC8434R",
               "custtype": "P",
               }
    params = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "AFHR_FLPR_YN": "N",
        "OFL_YN": "",
        "INQR_DVSN": "02",
        "UNPR_DVSN": "01",
        "FUND_STTL_ICLD_YN": "N",
        "FNCG_AMT_AUTO_RDPT_YN": "N",
        "PRCS_DVSN": "01",
        "CTX_AREA_FK100": "",
        "CTX_AREA_NK100": ""
    }
    res = requests.get(URL, headers=headers, params=params)
    stock_list = res.json()['output1']
    evaluation = res.json()['output2']
    stock_dict = {}
    send_message(f"====주식 보유잔고====")
    for stock in stock_list:
        if int(stock['hldg_qty']) > 0:
            stock_dict[stock['pdno']] = stock['hldg_qty']
            send_message(
                f"{stock['prdt_name']}({stock['pdno']}): {stock['hldg_qty']}주")
            time.sleep(0.1)
    send_message(f"주식 평가 금액: {evaluation[0]['scts_evlu_amt']}원")
    time.sleep(0.1)
    send_message(f"평가 손익 합계: {evaluation[0]['evlu_pfls_smtl_amt']}원")
    time.sleep(0.1)
    send_message(f"총 평가 금액: {evaluation[0]['tot_evlu_amt']}원")
    time.sleep(0.1)
    send_message(f"=================")
    return stock_dict

In [9]:
def get_balance():
    """현금 잔고조회"""
    PATH = "uapi/domestic-stock/v1/trading/inquire-psbl-order"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type": "application/json",
               "authorization": f"Bearer {ACCESS_TOKEN}",
               "appKey": APP_KEY,
               "appSecret": APP_SECRET,
               "tr_id": "TTTC8908R",
               "custtype": "P",
               }
    params = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "PDNO": "005930",
        "ORD_UNPR": "65500",
        "ORD_DVSN": "01",
        "CMA_EVLU_AMT_ICLD_YN": "Y",
        "OVRS_ICLD_YN": "Y"
    }
    res = requests.get(URL, headers=headers, params=params)
    cash = res.json()['output']['ord_psbl_cash']
    send_message(f"주문 가능 현금 잔고: {cash}원")
    return int(cash)

In [10]:
def buy(code, qty):
    """주식 시장가 매수"""
    PATH = "uapi/domestic-stock/v1/trading/order-cash"
    URL = f"{URL_BASE}/{PATH}"
    data = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "PDNO": code,
        "ORD_DVSN": "01",
        "ORD_QTY": str(int(qty)),
        "ORD_UNPR": "0",
    }
    headers = {"Content-Type": "application/json",
               "authorization": f"Bearer {ACCESS_TOKEN}",
               "appKey": APP_KEY,
               "appSecret": APP_SECRET,
               #"tr_id": "TTTC0802U",
               "tr_id":"VTTC0802U",
               "custtype": "P",
               "hashkey": hashkey(data)
               }
    res = requests.post(URL, headers=headers, data=json.dumps(data))
    if res.json()['rt_cd'] == '0':
        send_message(f"[매수 성공]{str(res.json())}")
        return True
    else:
        send_message(f"[매수 실패]{str(res.json())}")
        return False

In [20]:
code_list = input("모의투자 진행을 원하시는 종목 이름를 개수와 상관없이 입력하시오: ").split()
code_list

모의투자 진행을 원하시는 종목 이름를 개수와 상관없이 입력하시오: 005930 019170 035720


['005930', '019170', '035720']

In [21]:
#map(int, input().split())
buy_qty = list(input("종목코드에 따른 수량을 순서대로 적으시오: ").split())
#buy_qty = input("종목코드에 따른 수량(순서대로): ").split()
buy_qty

종목코드에 따른 수량을 순서대로 적으시오: 1 2 3


['1', '2', '3']

In [28]:
# 자동매매 시작
try:
    #API 사용을 위해 발급받는 토큰
    ACCESS_TOKEN = get_access_token()
    
    bought_list = [] # 매수 완료된 종목 리스트

    target_buy_count = 100 # 매수가능 수
    buy_percent = 0.33 # 보유한 전체 현금에서 몇%를 한 종목당 매수 금액으로 사용할 것인지에 대한 비율

    select = input('당신의 위험 회피성향은?')
    
    send_message(f"===국내 주식 자동매매 프로그램을 시작합니다===")
    t_now = datetime.datetime.now()
    t_start = t_now.replace(hour=9, minute=0, second=0, microsecond=0)
    t_exit = t_now.replace(hour=15, minute=30, second=0,microsecond=0)
    today = datetime.datetime.today().weekday()
    if today == 5 or today == 6:  # 토요일이나 일요일이면 프로그램 자동 종료
        send_message("주말이므로 매매가 불가능합니다.")
    if t_start < t_now < t_exit :
        for code in code_list:
            if len(bought_list) < target_buy_count:
                #buy_qty = 1
                target_price = get_target_price(code, select)# 목표 가격
                current_price = get_current_price(code)# 현재 가격
                    
                if target_price < current_price: # 목표 가격보다 현재 가격이 더 크면 매수 추천
                    for buyNum in buy_qty:
                        send_message(f"{code} 목표가 달성({target_price} < {current_price}) 매수를 시도합니다.")
                        result = buy(sym, buyNum)
                    if result:
                        soldout = False
                else:
                    send_message(f"{code} 목표가 달성 실패({target_price} > {current_price}) 매수를 실패하였습니다.")
                    if result:
                        soldout = False
                time.sleep(1)
        time.sleep(1)
    if (t_exit < t_now)|(t_start > t_now):  # PM 03:30 ~ :프로그램 종료
        send_message("주식 장이 마감되었습니다.")    

except Exception as e:
    send_message(f"오류발생.", e)
    time.sleep(1)


당신의 위험 회피성향은?0
{'content': '[2022-11-16 15:54:47] ===국내 주식 자동매매 프로그램을 시작합니다==='}
{'content': '[2022-11-16 15:54:48] 주식 장이 마감되었습니다.'}


In [27]:
VAutoTrade_info = []

VAutoTrade_info.append(code_list) # 종목
VAutoTrade_info.append(buy_qty) #종목 당 매수 개수
VAutoTrade_info.append(target_price) # 목표가
VAutoTrade_info.append(current_price) # 현재가

print(VAutoTrade_info)

[['028670', '019170', '011810', '005930'], ['1', '2', '3', '4'], 28308.5, 30300]


In [25]:
VAT_dict = {"VAutoTrade":{"stk_cd":code_list, "stk_buyN":buy_qty, "target_price":target_price, "current_price":current_price}}
print(VAT_dict)

{'VAutoTrade': {'stk_cd': ['005930', '019170', '035720'], 'stk_buyN': ['1', '2', '3'], 'target_price': 62850.0, 'current_price': 62700}}


In [27]:
import json

VAT_json = json.dumps(VAT_dict, indent=4)
print(VAT_json)

{
    "VAutoTrade": {
        "stk_cd": [
            "005930",
            "019170",
            "035720"
        ],
        "stk_buyN": [
            "1",
            "2",
            "3"
        ],
        "target_price": 62850.0,
        "current_price": 62700
    }
}
